In [ ]:
import pygame
import random
from typing import List, Tuple, Optional
import math

# =========================
# KONFIG (windowed / responsive)
# =========================
FPS = 60
TOP_GAP = 56          # tinggi top bar
BOTTOM_H = 80         # tinggi bottom panel
MARGIN = 3            # jarak antar sel (referensi; bisa disesuaikan per board)
CELL_SIZE = 64        # ukuran sel referensi (dipakai sebagai maksimum)
ANIM_DURATION_SEC = 0.15
AI_DELAY_MS = 400

# Level list: (name, grid_size, coins)
# koin semakin sedikit saat level semakin sulit
LEVELS = [
    ("Easy",   5, 12),   # grid 5x5, banyak koin
    ("Normal", 7, 8),    # grid 7x7, jumlah koin sedang
    ("Hard",   9, 5),    # grid 9x9, sedikit koin (lebih sulit)
]

# =========================
# WARNA
# =========================
BG = (20, 20, 24)
CELL_BG = (40, 40, 48)

COIN_COLOR   = (240, 210, 60)
COIN_OUTLINE = (180, 150, 40)
COIN_SHINE   = (255, 255, 220)

GOLD = (255, 215, 0)   # warna emas

HUMAN_COLOR = (90, 200, 250)    # biru
AI_COLOR    = (255, 100, 120)   # merah
SHADOW_COLOR = (25, 25, 28)

TEXT    = (235, 235, 240)
SUBTEXT = (190, 195, 205)
TOPBAR_BG  = (30, 32, 38)
PANEL_BG   = (30, 30, 38)
BLACK = (0, 0, 0)

WIN_COLORS = [(255,100,120), (90,200,250), (240,210,60), (100,255,100), (255,150,255)]

# =========================
# VAR GLOBAL (responsive window)
# =========================
# WINDOW size (actual window). Jangan confuse dengan grid size.
WIN_W = 640
WIN_H = 800

GRID_SIZE = 7
LEVEL_COINS = 11

def apply_level(grid_size: int, coins: int):
    """Set ukuran board & coins (tidak memaksa ukuran window)."""
    global GRID_SIZE, LEVEL_COINS
    GRID_SIZE = grid_size
    LEVEL_COINS = coins
    # window size left to user (resizable) - board will scale to fit

# =========================
# TIPE
# =========================
Vec2  = Tuple[int, int]
Vec2f = Tuple[float, float]

# =========================
# UTIL
# =========================
def clamp(n, lo, hi): return max(lo, min(hi, n))

def manhattan(a: Vec2, b: Vec2) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def is_adjacent(a: Vec2, b: Vec2) -> bool:
    return manhattan(a, b) == 1

def lerp(a: float, b: float, t: float) -> float:
    return a + (b - a) * t

def nearest_coin(from_pos: Vec2, coins: List[Vec2]) -> Optional[Vec2]:
    if not coins:
        return None
    return min(coins, key=lambda c: manhattan(from_pos, c))

# =========================
# BOARD METRICS (RESPONSIVE)
# =========================
def get_board_metrics():
   
    global WIN_W, WIN_H, GRID_SIZE
    # Available area untuk board (di antara top gap dan panel bawah)
    available_w = WIN_W
    available_h = WIN_H - TOP_GAP - BOTTOM_H
    # gunakan sisi terkecil agar muat
    available_side = min(available_w, available_h)
    base_margin = MARGIN
    if GRID_SIZE <= 0:
        cs = CELL_SIZE
    else:
        cs = int((available_side - (GRID_SIZE + 1) * base_margin) / GRID_SIZE)
        if cs <= 8:
            cs = 8  # minimal agar masih bisa tampil
        cs = min(cs, CELL_SIZE)  # jangan lebih besar dari referensi CELL_SIZE

    total_board_pixels = GRID_SIZE * cs + (GRID_SIZE + 1) * base_margin
    ox = (WIN_W - total_board_pixels) // 2
    oy = TOP_GAP + (available_h - total_board_pixels) // 2
    return cs, base_margin, ox, oy

# Ukuran board (px) and origin using metrics
def board_size_px() -> int:
    cs, m, _, _ = get_board_metrics()
    return GRID_SIZE * cs + (GRID_SIZE + 1) * m

def board_origin() -> Tuple[int, int]:
    _, _, ox, oy = get_board_metrics()
    return ox, oy

# Grid<->Pixel (memakai metrics)
def grid_to_px(cell: Vec2) -> Vec2f:
    cs, m, ox, oy = get_board_metrics()
    x, y = cell
    px = ox + m + x * (cs + m)
    py = oy + m + y * (cs + m)
    return (px, py)

def cell_from_mouse(mx: int, my: int) -> Optional[Vec2]:
    cs, m, ox, oy = get_board_metrics()
    bw = GRID_SIZE * cs + (GRID_SIZE + 1) * m
    if not (ox <= mx < ox + bw and oy <= my < oy + bw):
        return None
    gx = (mx - ox - m) // (cs + m)
    gy = (my - oy - m) // (cs + m)
    if 0 <= gx < GRID_SIZE and 0 <= gy < GRID_SIZE:
        return (int(gx), int(gy))
    return None

def step_towards(src: Vec2, dst: Vec2) -> Vec2:
    x, y = src
    dx = dst[0] - x
    dy = dst[1] - y
    if abs(dx) >= abs(dy):
        x += 1 if dx > 0 else (-1 if dx < 0 else 0)
    else:
        y += 1 if dy > 0 else (-1 if dy < 0 else 0)
    x = clamp(x, 0, GRID_SIZE - 1)
    y = clamp(y, 0, GRID_SIZE - 1)
    return (x, y)

# =========================
# CLASS ROUND (1 level)
# =========================
class RoundGame:
    """
    State: HUMAN_TURN, HUMAN_MOVING, AI_TURN, AI_MOVING, OVER
    """
    def __init__(self):
        self.reset()

    @property
    def human_start(self) -> Vec2: return (0, GRID_SIZE - 1)
    @property
    def ai_start(self)    -> Vec2: return (GRID_SIZE - 1, 0)

    def _spawn_coins(self) -> List[Vec2]:
        taken = {self.human, self.ai}
        coins = set()
        # uses LEVEL_COINS set by apply_level
        while len(coins) < LEVEL_COINS:
            c = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
            if c not in taken and c not in coins:
                coins.add(c)
        return sorted(list(coins))

    def reset(self):
        # spawn awal (pastikan berbeda)
        self.human = self.human_start
        self.ai    = self.ai_start
        if self.human == self.ai:
            # seandainya grid kecil ekstrem—acak ulang AI hingga beda
            while True:
                candidate = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
                if candidate != self.human:
                    self.ai = candidate
                    break

        # posisi pixel (anim)
        self.human_px = grid_to_px(self.human)
        self.ai_px    = grid_to_px(self.ai)
        self.human_start_px = self.human_px
        self.ai_start_px    = self.ai_px
        self.human_target_grid = self.human
        self.ai_target_grid    = self.ai

        self.anim_timer = 0.0

        self.coins: List[Vec2] = self._spawn_coins()
        self.human_score = 0
        self.ai_score = 0

        self.state = "HUMAN_TURN"
        self.over = False
        self.result = None          # "HUMAN_WIN" / "AI_WIN" / "DRAW"

    # ---- logika skor/akhir ----
    def _collect_if_any(self, who: str):
        pos = self.human if who == "HUMAN" else self.ai
        if pos in self.coins:
            self.coins.remove(pos)
            if who == "HUMAN": self.human_score += 1
            else:              self.ai_score += 1

    def _check_over(self):
        if not self.coins:
            self.over = True
            if self.human_score > self.ai_score: self.result = "HUMAN_WIN"
            elif self.ai_score > self.human_score: self.result = "AI_WIN"
            else: self.result = "DRAW"

    # ---- Human & AI move (dengan collision handling) ----
    def human_try_move(self, target_cell: Vec2) -> bool:
        if self.state != "HUMAN_TURN":
            return False
        if not (0 <= target_cell[0] < GRID_SIZE and 0 <= target_cell[1] < GRID_SIZE):
            return False
        if not is_adjacent(self.human, target_cell):
            return False
        if target_cell == self.ai:
            # tidak boleh menabrak/memasuki sel AI
            return False

        self.human_start_px = self.human_px
        self.human_target_grid = target_cell
        self.anim_timer = 0.0
        self.state = "HUMAN_MOVING"
        return True

    def ai_start_move(self):
        if self.state != "AI_TURN" or self.over:
            return
        if not self.coins:
            self._check_over(); return

        target_coin = nearest_coin(self.ai, self.coins)
        if target_coin is None:
            self._check_over(); return

        # rencana langkah 1 sel menuju koin
        candidate = step_towards(self.ai, target_coin)

        # === ERROR HANDLING POSISI ===
        # AI tidak boleh masuk sel Human
        if candidate == self.human:
            # coba alternatif: memprioritaskan sumbu lain
            ax, ay = self.ai
            tx, ty = target_coin
            options: List[Vec2] = []
            # dua kemungkinan axis
            if tx != ax:
                options.append((ax + (1 if tx > ax else -1), ay))
            if ty != ay:
                options.append((ax, ay + (1 if ty > ay else -1)))
            # filter valid dan bukan posisi human
            options = [(x, y) for (x, y) in options
                       if 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and (x, y) != self.human]
            # jika ada opsi aman, pilih yang paling mendekatkan koin
            if options:
                candidate = min(options, key=lambda p: manhattan(p, target_coin))
            else:
                # tidak ada langkah aman—AI skip turn
                return

        self.ai_start_px = self.ai_px
        self.ai_target_grid = candidate
        self.anim_timer = 0.0
        self.state = "AI_MOVING"

    def update_animation(self, dt_sec: float):
        if self.state == "HUMAN_MOVING":
            self.anim_timer += dt_sec
            t = min(1.0, self.anim_timer / ANIM_DURATION_SEC)
            start_px = self.human_start_px
            end_px   = grid_to_px(self.human_target_grid)
            self.human_px = (lerp(start_px[0], end_px[0], t), lerp(start_px[1], end_px[1], t))
            if t >= 1.0:
                # commit grid
                self.human = self.human_target_grid
                self.human_px = end_px
                self._collect_if_any("HUMAN")
                self._check_over()
                if not self.over:
                    self.state = "AI_TURN"

        elif self.state == "AI_MOVING":
            self.anim_timer += dt_sec
            t = min(1.0, self.anim_timer / ANIM_DURATION_SEC)
            start_px = self.ai_start_px
            end_px   = grid_to_px(self.ai_target_grid)
            self.ai_px = (lerp(start_px[0], end_px[0], t), lerp(start_px[1], end_px[1], t))
            if t >= 1.0:
                self.ai = self.ai_target_grid
                self.ai_px = end_px
                self._collect_if_any("AI")
                self._check_over()
                if not self.over:
                    self.state = "HUMAN_TURN"

# =========================
# RENDER
# =========================
def draw_topbar(screen, font, font_small, level_name: str):
    pygame.draw.rect(screen, TOPBAR_BG, (0, 0, WIN_W, TOP_GAP))
    lbl = font.render(f"Round – {level_name}", True, TEXT)
    screen.blit(lbl, (16, TOP_GAP//2 - lbl.get_height()//2))

    # Legend kanan
    hx = WIN_W - 260; hy = TOP_GAP//2
    pygame.draw.circle(screen, HUMAN_COLOR, (hx, hy), 10)
    lab_h = font_small.render("Human", True, TEXT)
    screen.blit(lab_h, (hx + 14, hy - lab_h.get_height()//2))

    ax = WIN_W - 140; ay = TOP_GAP//2
    pygame.draw.rect(screen, AI_COLOR, (ax-10, ay-10, 20, 20), border_radius=4)
    lab_ai = font_small.render("AI", True, TEXT)
    screen.blit(lab_ai, (ax + 14, ay - lab_ai.get_height()//2))

def draw_grid(screen):
    screen.fill(BG)
    cs, m, ox, oy = get_board_metrics()
    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            px = ox + m + x * (cs + m)
            py = oy + m + y * (cs + m)
            pygame.draw.rect(screen, CELL_BG, pygame.Rect(px, py, cs, cs), border_radius=4)

def draw_coins(screen, coins: List[Vec2], bob_offset: float):
    cs, m, ox, oy = get_board_metrics()
    for c in coins:
        px = ox + m + c[0] * (cs + m)
        py = oy + m + c[1] * (cs + m)
        cx = int(px + cs // 2)
        cy = int(py + cs // 2 + bob_offset)
        r = max(4, int(cs * 0.25))
        pygame.draw.circle(screen, COIN_OUTLINE, (cx, cy), r + 2)
        pygame.draw.circle(screen, COIN_COLOR,  (cx, cy), r)
        shine_r = max(2, r // 2)
        pygame.draw.circle(screen, COIN_SHINE, (cx - shine_r//2, cy - shine_r//2), shine_r)

def draw_human(screen, px_pos: Vec2f):
    cs, m, ox, oy = get_board_metrics()
    px, py = px_pos
    cx = int(px + cs // 2)
    cy = int(py + cs // 2)
    r = max(6, int(cs * 0.45))
    pygame.draw.circle(screen, SHADOW_COLOR, (cx, cy + max(1, r//6)), r)
    pygame.draw.circle(screen, HUMAN_COLOR, (cx, cy), r)
    eye_r = max(2, int(cs * 0.08))
    eye_y = cy - r // 3
    eye_x_off = r // 2
    pygame.draw.circle(screen, BLACK, (cx - eye_x_off, eye_y), eye_r)
    pygame.draw.circle(screen, BLACK, (cx + eye_x_off, eye_y), eye_r)
    smile_y = cy + r // 3
    smile_w = int(r / 1.5)
    pygame.draw.line(screen, BLACK, (cx - smile_w // 2, smile_y), (cx + smile_w // 2, smile_y), max(1, int(cs * 0.06)))

def draw_ai(screen, px_pos: Vec2f):
    cs, m, ox, oy = get_board_metrics()
    px, py = px_pos
    pad = max(2, int(cs * 0.12))
    shadow = pygame.Rect(px + pad, py + pad + max(1, int(cs * 0.06)), cs - 2*pad, cs - 2*pad)
    pygame.draw.rect(screen, SHADOW_COLOR, shadow, border_radius=max(2, int(cs * 0.08)))
    rect = pygame.Rect(px + pad, py + pad, cs - 2*pad, cs - 2*pad)
    pygame.draw.rect(screen, AI_COLOR, rect, border_radius=max(2, int(cs * 0.08)))
    eye = max(4, int(cs * 0.16)); ep = max(4, int(cs * 0.16))
    ey = py + pad + ep; ex1 = px + pad + ep; ex2 = px + cs - pad - ep - eye
    pygame.draw.rect(screen, BLACK, (ex1, ey, eye, eye))
    pygame.draw.rect(screen, BLACK, (ex2, ey, eye, eye))
    my = py + cs - pad - ep
    pygame.draw.line(screen, BLACK, (ex1, my), (ex2 + eye, my), max(1, int(cs * 0.06)))

def draw_panel(screen, font_big, font_small, rg: RoundGame):
    py = WIN_H - BOTTOM_H
    pygame.draw.rect(screen, PANEL_BG, (0, py, WIN_W, BOTTOM_H))
    # skor
    t_h = font_big.render(f"Human: {rg.human_score}", True, TEXT)
    screen.blit(t_h, (16, py + 10))
    t_a = font_big.render(f"AI: {rg.ai_score}", True, TEXT)
    screen.blit(t_a, (WIN_W - t_a.get_width() - 16, py + 10))

    # turn / hasil
    if not rg.over:
        if rg.state in ("HUMAN_TURN", "HUMAN_MOVING"):
            col = HUMAN_COLOR; label = "Your Turn"
        elif rg.state in ("AI_TURN", "AI_MOVING"):
            col = AI_COLOR; label = "AI Turn"
        else:
            col = SUBTEXT; label = "—"
        big = font_big.render(label, True, col)
        screen.blit(big, (WIN_W//2 - big.get_width()//2, py + 8))
    else:
        label = "You Win!" if rg.result=="HUMAN_WIN" else ("You Lose" if rg.result=="AI_WIN" else "Draw")
        col = HUMAN_COLOR if rg.result=="HUMAN_WIN" else (AI_COLOR if rg.result=="AI_WIN" else SUBTEXT)
        big = font_big.render(label, True, col)
        screen.blit(big, (WIN_W//2 - big.get_width()//2, py + 8))

    hint = font_small.render("Use Arrow to move   •   Press R to Restart", True, SUBTEXT)
    screen.blit(hint, (WIN_W//2 - hint.get_width()//2, py + 46))

# =========================
# MENU & END SCREENS
# =========================
def draw_button(screen, rect: pygame.Rect, text: str, font, hover: bool):
    bg = (70,74,86) if hover else (50,54,62)
    pygame.draw.rect(screen, bg, rect, border_radius=10)
    lbl = font.render(text, True, TEXT)
    screen.blit(lbl, (rect.centerx - lbl.get_width()//2, rect.centery - lbl.get_height()//2))

def draw_menu(screen, font_title, font):
    screen.fill(BG)
    t = font_title.render("COIN HUNT", True, GOLD)
    screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//3 - 40))
    s = font.render("Get Ready to Start", True, SUBTEXT)
    screen.blit(s, (WIN_W//2 - s.get_width()//2, WIN_H//3 + 24))
    w,h = 260, 56
    rect = pygame.Rect(WIN_W//2 - w//2, WIN_H//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Click to Begin", font, hover)
    return rect

def draw_level_select(screen, font_title, font):
    screen.fill(BG)
    t = font_title.render("Select Level", True, GOLD)
    screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//6))
    # level buttons
    w,h = 260,56
    gap = 18
    start_y = WIN_H//3
    rects = []
    for i, (name, _, coins) in enumerate(LEVELS):
        label = f"{name}"
        rect = pygame.Rect(WIN_W//2 - w//2, start_y + i*(h+gap), w, h)
        hover = rect.collidepoint(pygame.mouse.get_pos())
        draw_button(screen, rect, label, font, hover)
        rects.append(rect)
    return rects

def draw_round_select(screen, font_title, font, current_round):
    screen.fill(BG)
    t = font_title.render("Select Rounds", True, GOLD)
    screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//6))
    # minus, number, plus
    minus = pygame.Rect(WIN_W//2 - 170, WIN_H//2 - 28, 100, 56)
    plus  = pygame.Rect(WIN_W//2 + 70, WIN_H//2 - 28, 100, 56)
    num_r = pygame.Rect(WIN_W//2 - 60, WIN_H//2 - 28, 120, 56)
    hover_m = minus.collidepoint(pygame.mouse.get_pos())
    hover_p = plus.collidepoint(pygame.mouse.get_pos())
    hover_n = num_r.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, minus, "-", font, hover_m)
    draw_button(screen, num_r, str(current_round), font, hover_n)
    draw_button(screen, plus, "+", font, hover_p)
    start_rect = pygame.Rect(WIN_W//2 - 140, WIN_H//2 + 80, 280, 64)
    hover_start = start_rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, start_rect, "Start Game", font, hover_start)
    return minus, num_r, plus, start_rect

def draw_next_round(screen, font_big, font):
    screen.fill(BG)
    t = font_big.render("Round Complete!", True, HUMAN_COLOR)
    screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//3 - 24))
    s = font.render("Continue to next round", True, TEXT)
    screen.blit(s, (WIN_W//2 - s.get_width()//2, WIN_H//3 + 24))
    w,h = 320,56
    rect = pygame.Rect(WIN_W//2 - w//2, WIN_H//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Next Round", font, hover)
    return rect

def draw_end(screen, font_big, font, win: bool):
    screen.fill(BG)
    title = "You Beat All Rounds!" if win else "You Lost The Game"
    col = HUMAN_COLOR if win else AI_COLOR
    t = font_big.render(title, True, col)
    screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//3 - 24))
    s = font.render("Restart Game", True, TEXT)
    screen.blit(s, (WIN_W//2 - s.get_width()//2, WIN_H//3 + 24))
    w,h = 220,56
    rect = pygame.Rect(WIN_W//2 - w//2, WIN_H//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Play Again", font, hover)
    return rect

# =========================
# MAIN
# =========================
def main():
    pygame.init()
    random.seed()

    global WIN_W, WIN_H

    # mulai default Easy
    level_idx = 0
    name, size, coins = LEVELS[level_idx]
    apply_level(size, coins)

    # create resizable window; start with a reasonable size based on display
    info = pygame.display.Info()
    display_w, display_h = info.current_w, info.current_h
    # start window at ~60% of display height if possible
    start_h = min(900, max(600, int(display_h * 0.75)))
    start_w = int(start_h * 0.78)
    WIN_W, WIN_H = start_w, start_h
    screen = pygame.display.set_mode((WIN_W, WIN_H), pygame.RESIZABLE)
    pygame.display.set_caption("COIN HUNT — Human vs AI")

    # fonts
    font_title = pygame.font.SysFont("bahnschrift", 48)
    font_big   = pygame.font.SysFont("bahnschrift", 32)
    font       = pygame.font.SysFont("bahnschrift", 22)
    font_small = pygame.font.SysFont("bahnschrift", 18)

    clock = pygame.time.Clock()
    running = True

    # state global: MENU, LEVEL_SELECT, ROUND_SELECT, PLAYING, ROUND_CLEAR, GAME_OVER
    state = "MENU"
    rg: Optional[RoundGame] = None
    ai_timer = AI_DELAY_MS
    bob_timer = 0.0

    # round system
    total_rounds = 3
    current_round = 1
    human_wins = 0
    ai_wins = 0
    last_round_winner: Optional[str] = None  # "HUMAN"/"AI"/"DRAW"

    while running:
        dt_ms = clock.tick(FPS)
        dt = dt_ms / 1000.0
        bob_timer = (bob_timer + dt * 5.0) % (math.pi * 2)
        bob_offset = math.sin(bob_timer) * 3.0

        # --- EVENT ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # handle window resize
            elif event.type == pygame.VIDEORESIZE:
                WIN_W, WIN_H = max(320, event.w), max(400, event.h)
                screen = pygame.display.set_mode((WIN_W, WIN_H), pygame.RESIZABLE)

            # ---------------------------
            # MENU: Click to Begin -> Level Select
            # ---------------------------
            elif state == "MENU" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # go to level select
                state = "LEVEL_SELECT"

            # ---------------------------
            # LEVEL_SELECT: determine which level rect was clicked
            # ---------------------------
            elif state == "LEVEL_SELECT" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # build same rects as drawn to test collision
                w,h = 260,56
                gap = 18
                start_y = WIN_H//3
                for i, (lname, lsize, lcoins) in enumerate(LEVELS):
                    rect = pygame.Rect(WIN_W//2 - w//2, start_y + i*(h+gap), w, h)
                    if rect.collidepoint(event.pos):
                        level_idx = i
                        name, size, coins = LEVELS[level_idx]
                        apply_level(size, coins)
                        # reset round system
                        total_rounds = 3
                        current_round = 1
                        human_wins = 0
                        ai_wins = 0
                        last_round_winner = None
                        # create a RoundGame for first round
                        rg = RoundGame()
                        state = "ROUND_SELECT"
                        break

            # ---------------------------
            # ROUND_SELECT: plus, minus, start game
            # ---------------------------
            elif state == "ROUND_SELECT" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                minus = pygame.Rect(WIN_W//2 - 170, WIN_H//2 - 28, 100, 56)
                plus  = pygame.Rect(WIN_W//2 + 70, WIN_H//2 - 28, 100, 56)
                start_rect = pygame.Rect(WIN_W//2 - 140, WIN_H//2 + 80, 280, 64)
                if minus.collidepoint(event.pos):
                    if total_rounds > 1:
                        total_rounds -= 1
                elif plus.collidepoint(event.pos):
                    if total_rounds < 9:
                        total_rounds += 1
                elif start_rect.collidepoint(event.pos):
                    # Start first round
                    current_round = 1
                    human_wins = 0
                    ai_wins = 0
                    last_round_winner = None
                    # ensure RoundGame exists and is reset
                    if rg is None:
                        rg = RoundGame()
                    else:
                        rg.reset()
                    state = "PLAYING"
                    ai_timer = AI_DELAY_MS

            # ---------------------------
            # PLAYING: input handling (keystrokes/mouse)
            # ---------------------------
            elif state == "PLAYING" and event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    if rg:
                        rg.reset()
                elif rg and rg.state == "HUMAN_TURN":
                    cx, cy = rg.human
                    if event.key in (pygame.K_w, pygame.K_UP):
                        rg.human_try_move((cx, cy - 1))
                    elif event.key in (pygame.K_s, pygame.K_DOWN):
                        rg.human_try_move((cx, cy + 1))
                    elif event.key in (pygame.K_a, pygame.K_LEFT):
                        rg.human_try_move((cx - 1, cy))
                    elif event.key in (pygame.K_d, pygame.K_RIGHT):
                        rg.human_try_move((cx + 1, cy))
            elif state == "PLAYING" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                if rg and rg.state == "HUMAN_TURN":
                    cell = cell_from_mouse(*event.pos)
                    if cell is not None:
                        rg.human_try_move(cell)

            # ---------------------------
            # ROUND_CLEAR: Next Round click + Back to Menu
            # ---------------------------
            elif state == "ROUND_CLEAR" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # Next round button rect (same as draw_next_round)
                rect = pygame.Rect(WIN_W//2 - 320//2, WIN_H//2, 320, 56)
                home_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2 + 80, 220, 56)  # Back to Menu
                if rect.collidepoint(event.pos):
                    # move to next round or end
                    current_round += 1
                    if current_round > total_rounds:
                        state = "GAME_OVER"
                    else:
                        # reset RoundGame for next round (same level)
                        if rg:
                            rg.reset()
                        last_round_winner = None
                        state = "PLAYING"
                elif home_rect.collidepoint(event.pos):
                    # back to main menu
                    state = "MENU"

            # ---------------------------
            # GAME_OVER: Play Again or Back to Home
            # ---------------------------
            elif state == "GAME_OVER" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # Play Again rect & Back to Menu rect (we will compute same positions as drawing)
                play_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2, 220, 56)
                home_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2 + 80, 220, 56)
                if play_rect.collidepoint(event.pos):
                    # restart same level: go to ROUND_SELECT with default 3
                    total_rounds = 3
                    current_round = 1
                    human_wins = 0
                    ai_wins = 0
                    last_round_winner = None
                    if rg:
                        rg.reset()
                    state = "ROUND_SELECT"
                elif home_rect.collidepoint(event.pos):
                    state = "MENU"

        # === UPDATE GAME STATE WHEN PLAYING ===
        if state == "PLAYING" and rg:
            # AI timer
            if rg.state == "AI_TURN" and not rg.over:
                ai_timer -= dt_ms
                if ai_timer <= 0:
                    rg.ai_start_move()
                    ai_timer = AI_DELAY_MS
            elif rg.state != "AI_TURN":
                ai_timer = AI_DELAY_MS

            rg.update_animation(dt)

            # round selesai?
            if rg.over:
                # determine round winner and increment counters
                if rg.result == "HUMAN_WIN":
                    human_wins += 1
                    last_round_winner = "HUMAN"
                elif rg.result == "AI_WIN":
                    ai_wins += 1
                    last_round_winner = "AI"
                else:
                    last_round_winner = "DRAW"

                # if already reached majority earlier, can finish early
                majority = (total_rounds // 2) + 1
                if human_wins >= majority or ai_wins >= majority or current_round >= total_rounds:
                    # all rounds done or majority reached -> GAME_OVER
                    state = "GAME_OVER"
                else:
                    # show round clear, let user press Next Round or Back to Menu
                    state = "ROUND_CLEAR"

        # === DRAW ===
        if state == "MENU":
            menu_rect = draw_menu(screen, font_title, font)

        elif state == "LEVEL_SELECT":
            level_rects = draw_level_select(screen, font_title, font)

        elif state == "ROUND_SELECT":
            minus_r, num_r, plus_r, start_r = draw_round_select(screen, font_title, font, total_rounds)
            # also show current selected level in header
            lvl_name = LEVELS[level_idx][0]
            small = font.render(f"Level: {lvl_name}", True, SUBTEXT)
            screen.blit(small, (16, TOP_GAP//2 - small.get_height()//2))

        elif state == "PLAYING" and rg:
            draw_grid(screen)
            draw_topbar(screen, font, font_small, LEVELS[level_idx][0])
            draw_coins(screen, rg.coins, bob_offset)
            draw_human(screen, rg.human_px)
            draw_ai(screen, rg.ai_px)
            draw_panel(screen, font_big, font_small, rg)

            # overlay: show round info and match score (top overlay)
            score_txt = font_big.render(f"Match: {human_wins} - {ai_wins}", True, TEXT)
            screen.blit(score_txt, (16, TOP_GAP + 8))
            round_txt = font_big.render(f"Round {current_round}/{total_rounds}", True, TEXT)
            screen.blit(round_txt, (WIN_W - round_txt.get_width() - 16, TOP_GAP + 8))

        elif state == "ROUND_CLEAR":
            # show who won this round + current match score
            screen.fill(BG)
            t = font_big.render(f"Round {current_round} Complete", True, TEXT)
            screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//3 - 24))
            # message per round
            if last_round_winner == "HUMAN":
                msg = "Round Winner: Human"
                col = HUMAN_COLOR
            elif last_round_winner == "AI":
                msg = "Round Winner: AI"
                col = AI_COLOR
            else:
                msg = "Round Result: Draw"
                col = SUBTEXT
            m = font.render(msg, True, col)
            screen.blit(m, (WIN_W//2 - m.get_width()//2, WIN_H//3 + 24))

            score = font.render(f"Match Score — Human {human_wins} : {ai_wins} AI", True, TEXT)
            screen.blit(score, (WIN_W//2 - score.get_width()//2, WIN_H//2 + 8))

            # Draw Next Round button and Back to Menu button
            rect = draw_next_round(screen, font_big, font)
            # back to menu button (same style as GAME_OVER)
            home_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2 + 80, 220, 56)
            hover_home = home_rect.collidepoint(pygame.mouse.get_pos())
            draw_button(screen, home_rect, "Back to Menu", font, hover_home)

        elif state == "GAME_OVER":
            screen.fill(BG)
            # final winner by total wins
            if human_wins > ai_wins:
                title = "You Win The Match!"
                col = HUMAN_COLOR
            elif ai_wins > human_wins:
                title = "AI Wins The Match!"
                col = AI_COLOR
            else:
                title = "Match Draw!"
                col = SUBTEXT
            t = font_big.render(title, True, col)
            screen.blit(t, (WIN_W//2 - t.get_width()//2, WIN_H//3 - 24))

            s = font.render(f"Final Score — Human {human_wins} : {ai_wins} AI", True, TEXT)
            screen.blit(s, (WIN_W//2 - s.get_width()//2, WIN_H//3 + 24))

            # Play Again (reselect rounds, same level) and Back to Home
            play_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2, 220, 56)
            hover_play = play_rect.collidepoint(pygame.mouse.get_pos())
            draw_button(screen, play_rect, "Play Again", font, hover_play)

            home_rect = pygame.Rect(WIN_W//2 - 220//2, WIN_H//2 + 80, 220, 56)
            hover_home = home_rect.collidepoint(pygame.mouse.get_pos())
            draw_button(screen, home_rect, "Back to Home", font, hover_home)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    # set initial level (Easy)
    apply_level(LEVELS[0][1], LEVELS[0][2])
    main()
